In [ ]:
%matplotlib inline
import numpy as np
import xarray as xr
from datetime import datetime

from itertools import product

import esmlab

import matplotlib.pyplot as plt

import config

In [ ]:
gridfile_directory = esmlab.config.get('regrid.gridfile-directory')
gridfile_directory

In [ ]:
def latlon_to_scrip(nx, ny, lon0=-180., grid_imask=None, file_out=None):
    """Generate a SCRIP grid file for a regular lat x lon grid.
    
    Parameters
    ----------
    
    nx : int
       Number of points in x (longitude).
    ny : int
       Number of points in y (latitude).
    lon0 : float, optional [default=-180]
       Longitude on lefthand grid boundary.
    grid_imask : array-like, optional [default=None]       
       If the value is set to 0 for a grid point, then that point is
       considered masked out and won't be used in the weights 
       generated by the application. 
    file_out : string, optional [default=None]
       File to which to write the grid.

    Returns
    -------
    
    ds : xarray.Dataset
       The grid file dataset.       
    """
    
    # compute coordinates of regular grid
    dx = 360. / nx
    dy = 180. / ny
    lat = np.arange(-90. + dy / 2., 90., dy)
    lon = np.arange(lon0 + dx / 2., lon0 + 360., dx)

    # make 2D
    y_center = np.broadcast_to(lat[:, None], (ny, nx))
    x_center = np.broadcast_to(lon[None, :], (ny, nx))

    # compute corner points: must be counterclockwise
    y_corner = np.stack((y_center - dy / 2.,  # SW
                         y_center - dy / 2.,  # SE
                         y_center + dy / 2.,  # NE
                         y_center + dy / 2.), # NW
                        axis=2)

    x_corner = np.stack((x_center - dx / 2.,  # SW
                         x_center + dx / 2.,  # SE
                         x_center + dx / 2.,  # NE
                         x_center - dx / 2.), # NW
                        axis=2)

    # compute area
    y0 = np.sin(y_corner[:, :, 0] * np.pi / 180.) # south
    y1 = np.sin(y_corner[:, :, 3] * np.pi / 180.) # north
    x0 = x_corner[:, :, 0] * np.pi / 180.         # west
    x1 = x_corner[:, :, 1] * np.pi / 180.         # east
    grid_area = (y1 - y0) * (x1 - x0)
    
    # sum of area should be equal to area of sphere
    np.testing.assert_allclose(grid_area.sum(), 4.*np.pi)
    
    # construct mask
    if grid_imask is None:
        grid_imask = np.ones((ny, nx), dtype=np.int32)
    
    # generate output dataset
    dso = xr.Dataset()    
    dso['grid_dims'] = xr.DataArray(np.array([nx, ny], dtype=np.int32), 
                                    dims=('grid_rank',)) 
    dso.grid_dims.encoding = {'dtype': np.int32}

    dso['grid_center_lat'] = xr.DataArray(y_center.reshape((-1,)), 
                                          dims=('grid_size'),
                                          attrs={'units': 'degrees'})

    dso['grid_center_lon'] = xr.DataArray(x_center.reshape((-1,)), 
                                          dims=('grid_size'),
                                          attrs={'units': 'degrees'})
    
    dso['grid_corner_lat'] = xr.DataArray(y_corner.reshape((-1, 4)), 
                                          dims=('grid_size', 'grid_corners'), 
                                          attrs={'units': 'degrees'})
    dso['grid_corner_lon'] = xr.DataArray(x_corner.reshape((-1, 4)), 
                                      dims=('grid_size', 'grid_corners'), 
                                      attrs={'units': 'degrees'})    

    dso['grid_imask'] = xr.DataArray(grid_imask.reshape((-1,)), 
                                     dims=('grid_size'),
                                     attrs={'units': 'unitless'})
    dso.grid_imask.encoding = {'dtype': np.int32}
    
    dso['grid_area'] = xr.DataArray(grid_area.reshape((-1,)), 
                                     dims=('grid_size'),
                                     attrs={'units': 'radians^2',
                                            'long_name': 'area weights'})
    
    # force no '_FillValue' if not specified
    for v in dso.variables:
        if '_FillValue' not in dso[v].encoding:
            dso[v].encoding['_FillValue'] = None

    dso.attrs = {'title': f'{dy} x {dx} (lat x lon) grid',
                 'created_by': 'latlon_to_scrip',
                 'date_created': f'{datetime.now()}',
                 'conventions': 'SCRIP',
                }
            
    # write output file
    if file_out is not None:
        print(f'writing {file_out}')
        dso.to_netcdf(file_out)
        
    return dso
    

Test grid file generation with very simple, low resolution grid.

In [ ]:
dso = latlon_to_scrip(nx=6, ny=9, lon0=-180.)
dims_grid = tuple(dso.grid_dims.values[::-1])
ny, nx = dims_grid

LON = dso.grid_center_lon.values.reshape(dims_grid)
LAT = dso.grid_center_lat.values.reshape(dims_grid)

x_corner = dso.grid_corner_lon.values.reshape(dims_grid+(4,))
y_corner = dso.grid_corner_lat.values.reshape(dims_grid+(4,))

ind = np.array([0, 1, 2, 3, 0])
for i, j in product(range(nx), range(ny)):
    p = plt.plot(LON[j, i], LAT[j, i], '.')
    plt.plot(x_corner[j, i, ind], y_corner[j, i, ind], 'x-', color=p[0].get_color())
    
dso.info()

In [ ]:
# check that this conforms to the grid of the original dataset
file_src_data = config.datasets['etopo1']['urlpath']
ds = xr.open_dataset(file_src_data)

grid_imask = xr.where(ds.z < 0, 1., 0.).values.astype(np.int32)

plt.pcolormesh(grid_imask[::10, ::10])
plt.colorbar()

Generate grid file for ETOPO1 dataset.

In [ ]:
%%time
file_out = f'{gridfile_directory}/etopo1.nc'
dso = latlon_to_scrip(nx=21600, ny=10800, lon0=-180., file_out=file_out)

# extract grid coordinates to make check below
dims_grid = tuple(dso.grid_dims.values[::-1])
ny, nx = dims_grid
lat = dso.grid_center_lat.values.reshape(dims_grid)[:, 0]
lon = dso.grid_center_lon.values.reshape(dims_grid)[0, :]

np.testing.assert_allclose(lat, ds.y.values)
np.testing.assert_allclose(lon, ds.x.values)

dso.info()

Generate remapping files.